# Pre-amble

Prior to the exploration, we first load some basic packages:

In [1]:
# loading the required packages
import pandas as pd
import numpy as np
import random
import re
import os
import plotnine
from plotnine import *

# for repeated printouts 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Basic dataset information

In [8]:
# loading the raw sentencing data 
sentencing_raw = pd.read_csv('../data/csv/sentencing.csv')

# printing the dataset characteristics
sentencing_raw.shape
sentencing_raw.info()
sentencing_raw.dtypes

# taking a look at the data head
sentencing_raw.head(n=10)

C:\Users\Sean Hambali\AppData\Local\Temp\ipykernel_16064\2838289327.py:2: DtypeWarning: Columns (10,11,14,25) have mixed types. Specify dtype option on import or set low_memory=False.


(272294, 41)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272294 entries, 0 to 272293
Data columns (total 41 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   CASE_ID                            272294 non-null  int64  
 1   CASE_PARTICIPANT_ID                272294 non-null  int64  
 2   RECEIVED_DATE                      272294 non-null  object 
 3   OFFENSE_CATEGORY                   272294 non-null  object 
 4   PRIMARY_CHARGE_FLAG                272294 non-null  bool   
 5   CHARGE_ID                          272294 non-null  int64  
 6   CHARGE_VERSION_ID                  272294 non-null  int64  
 7   DISPOSITION_CHARGED_OFFENSE_TITLE  272294 non-null  object 
 8   CHARGE_COUNT                       272294 non-null  int64  
 9   DISPOSITION_DATE                   272294 non-null  object 
 10  DISPOSITION_CHARGED_CHAPTER        272294 non-null  object 
 11  DISPOSITION_CHARGED_ACT            2668

CASE_ID                                int64
CASE_PARTICIPANT_ID                    int64
RECEIVED_DATE                         object
OFFENSE_CATEGORY                      object
PRIMARY_CHARGE_FLAG                     bool
CHARGE_ID                              int64
CHARGE_VERSION_ID                      int64
DISPOSITION_CHARGED_OFFENSE_TITLE     object
CHARGE_COUNT                           int64
DISPOSITION_DATE                      object
DISPOSITION_CHARGED_CHAPTER           object
DISPOSITION_CHARGED_ACT               object
DISPOSITION_CHARGED_SECTION           object
DISPOSITION_CHARGED_CLASS             object
DISPOSITION_CHARGED_AOIC              object
CHARGE_DISPOSITION                    object
CHARGE_DISPOSITION_REASON             object
SENTENCE_JUDGE                        object
SENTENCE_COURT_NAME                   object
SENTENCE_COURT_FACILITY               object
SENTENCE_PHASE                        object
SENTENCE_DATE                         object
SENTENCE_T

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,INCIDENT_CITY,INCIDENT_BEGIN_DATE,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
5,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
6,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242357787220,150253900073,ARMED ROBBERY,15,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
7,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242357787220,150253900073,ARMED ROBBERY,15,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
8,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242359023652,150251649536,ARMED ROBBERY,16,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion
9,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,True,1242194578090,151097726688,FIRST DEGREE MURDER,1,12/17/2014 12:00:00 AM,...,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion


# Data cleaning

## Cleaning demographic variables

## Cleaning datetime variables

## Cleaning sentencing-related variables

## Filtering to prepare analysis-ready dataset